In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Resources/worldbank_all-data.csv')

In [2]:
# Arrays for for loops
countries = ['ABW', 'ASM', 'BEL', 'BRB', 'AFG', 'BRN', 'ALB', 'AND', 'AGO',
       'CUW', 'DMA', 'GIN', 'GRD', 'ARG', 'ARM', 'ATG', 'AUT', 'AUS',
       'AZE', 'BDI', 'BFA', 'BGR', 'BHR', 'BHS', 'BLR', 'BLZ', 'BMU',
       'BEN', 'BGD', 'BTN', 'BIH', 'CAN', 'CHI', 'BOL', 'BRA', 'CHN',
       'CPV', 'CUB', 'CYM', 'BWA', 'CAF', 'DZA', 'HRV', 'KHM', 'TCD',
       'VGB', 'CHL', 'COD', 'CIV', 'COG', 'COM', 'COL', 'CYP', 'CZE',
       'DEU', 'CRI', 'DJI', 'DNK', 'ECU', 'ERI', 'FIN', 'FJI', 'FRA',
       'FRO', 'GIB', 'DOM', 'ETH', 'EGY', 'GNQ', 'EST', 'GRC', 'GRL',
       'GAB', 'GHA', 'GEO', 'GMB', 'GNB', 'GTM', 'GUM', 'HTI', 'JPN',
       'GUY', 'NOR', 'HKG', 'XKX', 'HND', 'HUN', 'IMN', 'IDN', 'IND',
       'IRL', 'IRQ', 'IRN', 'ISL', 'ISR', 'JAM', 'JOR', 'ITA', 'PYF',
       'SLV', 'SWZ', 'FSM', 'KAZ', 'KIR', 'KOR', 'LBN', 'LBR', 'LBY',
       'LIE', 'LSO', 'LTU', 'LUX', 'LVA', 'KEN', 'KGZ', 'MAC', 'MCO',
       'KWT', 'LAO', 'MDV', 'MEX', 'MHL', 'MAR', 'MLI', 'MMR', 'MNE',
       'MDA', 'MNP', 'MDG', 'MOZ', 'MWI', 'NAM', 'NCL', 'MKD', 'NER',
       'NGA', 'NIC', 'NLD', 'MLT', 'NPL', 'OMN', 'PRK', 'MNG', 'MRT',
       'MUS', 'KNA', 'LCA', 'MAF', 'MYS', 'NZL', 'PAK', 'ESP', 'LKA',
       'PAN', 'PER', 'PHL', 'PLW', 'SYC', 'PNG', 'POL', 'PRI', 'PRT',
       'QAT', 'PRY', 'RUS', 'SAU', 'SGP', 'SMR', 'ROU', 'RWA', 'SEN',
       'SLB', 'SOM', 'SLE', 'SSD', 'STP', 'SRB', 'SVK', 'SVN', 'SXM',
       'WSM', 'ARE', 'GBR', 'ZAF', 'SDN', 'SUR', 'SWE', 'TCA', 'CHE',
       'TKM', 'TLS', 'TON', 'TTO', 'TUV', 'SYR', 'USA', 'VCT', 'VEN',
       'VIR', 'TGO', 'TJK', 'VNM', 'THA', 'VUT', 'YEM', 'TUN', 'TZA',
       'UGA', 'UKR', 'URY', 'UZB', 'ZMB', 'ZWE']

years = ['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996','1997', '1998', '1999']

In [3]:
# Per-country .csv generation loop
for country in countries:

    # Create per-country dataframe
    df1 = df.loc[df['country_code'] == country]

    # Drop unnecessary columns
    df1.drop('country_name', axis=1, inplace=True)
    df1.drop('country_code', axis=1, inplace=True)
    df1.drop('indicator_name', axis=1, inplace=True)

    # Retain index pre-transposing
    df1.set_index('indicator_code', inplace=True)

    # Transpose matrix (rows are now years)
    dfT = df1.T

    # Drop 1960 to 1999
    for year in years:
        dfT = dfT.drop(year, axis=0)

    # Export per-country .csv
    dfT.to_csv('Countries/' + country + '.csv')

/Users/rmtmacbook/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [4]:
# Process country.csv and eliminate empty data
for country in countries:
    df = pd.read_csv('Countries/' + country + '.csv')

    # Replace NaN with str for readability
    df.fillna('BLANK', inplace=True)

    indicators = df.columns

    for indicator in indicators:
        blank_count = (df[indicator].value_counts(dropna=False)).to_dict()

        # Count str to retain columns with 90% data points
        if 'BLANK' in blank_count:
            if blank_count["BLANK"] > 2:
                df.drop(indicator, axis=1, inplace=True)
            else:
                pass
        else:
            pass
    
    # Replace str with NaN to calculate means
    df.replace ('BLANK', np.nan, inplace=True)
    df = df.mean(skipna=True).to_frame().T
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df.rename(index={0: country}, inplace=True)
    df.to_csv('Countries_adjusted/' + country + '.csv')

In [5]:
# Obtain all indicator codes from .csv's to merge into one master list
indicator_codes = []

for country in countries:
    df = pd.read_csv('Countries_adjusted/' + country + '.csv')

    indicators = df.columns

    for indicator in indicators:
        if(indicator in indicator_codes):
            pass
        else:
            indicator_codes.append(indicator)

In [6]:
# Create empty dataframe for population
df = pd.DataFrame(columns = indicator_codes, index = countries).drop('Unnamed: 0', axis=1)

# Scan through all spreadsheets and populate their information into the dataframe
for country in countries:
    df_c = pd.read_csv('Countries_adjusted/' + country + '.csv').drop('Unnamed: 0', axis=1).rename(index={0: country})
    for col in df_c.columns:
        df.loc[country, col] = df_c.loc[country, col]

In [7]:
# Utilize same algorithm to process master list
df.fillna('BLANK', inplace=True)

indicator_codes = df.columns

for indicator in indicator_codes:
    blank_count = (df[indicator].value_counts(dropna=False)).to_dict()

    # Still retaining 90% data points
    if 'BLANK' in blank_count:
        if blank_count["BLANK"] > 21:
            df.drop(indicator, axis=1, inplace=True)
        else:
            pass
    else:
        pass

df.replace ('BLANK', np.nan, inplace=True)
df.to_csv('Resources/worldbank_data.csv')
